# Трендовые признаки

- SMA, EMA, наклон MA, momentum
- Положение цены относительно средних
- Сигнал тренда (uptrend, downtrend, sideways)
- Confidence

Вся логика расчета трендовых метрик и визуализация распределения.


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("✅ Библиотеки загружены")


## Функции расчета трендовых признаков


In [ ]:
def sma(prices, window=20):
    """Simple Moving Average"""
    return prices.rolling(window=window).mean()


def ema(prices, span=20):
    """Exponential Moving Average"""
    return prices.ewm(span=span, adjust=False).mean()


def ma_slope(ma, window=5):
    """Наклон скользящей средней"""
    return (ma - ma.shift(window)) / window


def momentum(prices, window=10):
    """Momentum: изменение цены за период"""
    return prices - prices.shift(window)


def price_position(price, ma_short, ma_long):
    """Положение цены относительно средних: >1 = выше обеих, <-1 = ниже обеих"""
    above_short = (price > ma_short).astype(int)
    above_long = (price > ma_long).astype(int)
    return above_short + above_long - 1  # -1, 0, 1


def trend_signal(ma_short, ma_long, threshold=0.01):
    """
    Сигнал тренда:
    1 = uptrend (короткая MA выше длинной)
    -1 = downtrend (короткая MA ниже длинной)
    0 = sideways
    """
    diff = (ma_short - ma_long) / ma_long
    trend = np.where(diff > threshold, 1, np.where(diff < -threshold, -1, 0))
    return pd.Series(trend, index=ma_short.index)


def trend_confidence(ma_short, ma_long):
    """Уверенность в тренде: насколько далеко разошлись MA"""
    return np.abs(ma_short - ma_long) / ma_long

print("✅ Функции трендовых признаков загружены")


## Загрузка и расчет признаков


In [ ]:
# Загрузка данных
DATA_DIR = Path('data') / 'processed'
ticker = 'SBER'
df = pd.read_parquet(DATA_DIR / f"{ticker}_ohlcv_returns.parquet")

# Расчет скользящих средних
df['sma_20'] = sma(df['close'], window=20)
df['sma_50'] = sma(df['close'], window=50)
df['ema_20'] = ema(df['close'], span=20)
df['ema_50'] = ema(df['close'], span=50)

# Наклоны и моментум
df['sma_20_slope'] = ma_slope(df['sma_20'], window=5)
df['momentum_10'] = momentum(df['close'], window=10)
df['momentum_20'] = momentum(df['close'], window=20)

# Положение цены
df['price_position'] = price_position(df['close'], df['sma_20'], df['sma_50'])

# Сигнал тренда
df['trend_signal'] = trend_signal(df['sma_20'], df['sma_50'], threshold=0.01)
df['trend_confidence'] = trend_confidence(df['sma_20'], df['sma_50'])

print("✅ Трендовые признаки рассчитаны")
print(f"\nПример данных:")
print(df[['date', 'close', 'sma_20', 'sma_50', 'trend_signal', 'trend_confidence']].tail())


## Сохранение результатов


In [ ]:
OUTPUT_DIR = Path('data') / 'features'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

output_path = OUTPUT_DIR / f"{ticker}_trend_features.parquet"
df.to_parquet(output_path, index=False)
print(f"✅ Сохранено: {output_path}")
